In [1]:
!nvidia-smi

Thu Nov  5 21:44:25 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 435.21       Driver Version: 435.21       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce RTX 208...  Off  | 00000000:3B:00.0 Off |                  N/A |
| 28%   42C    P8    20W / 250W |      7MiB / 11019MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce RTX 208...  Off  | 00000000:5E:00.0 Off |                  N/A |
| 27%   35C    P8    20W / 250W |      7MiB / 11019MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   2  G

In [2]:
import json
import os
import numpy as np
import warnings
import matplotlib.pylab as plt
%matplotlib inline
warnings.filterwarnings("ignore")

#### This section is responsible for loading the face annotations.

In [3]:
FACE_ANNOTATIONS = "/home/firschj/src/CS548_Research_Code_GAN/Data/NVIDIA_StyleGAN2_Curated_Faces_sorted.json"

In [4]:
with open(FACE_ANNOTATIONS) as f:
    labels_data = json.load(f)
labels_data[0]

{'faceId': '800a28fe-d246-4da1-978d-10f112dad923',
 'faceRectangle': {'top': 331, 'left': 223, 'width': 575, 'height': 575},
 'faceLandmarks': {'pupilLeft': {'x': 394.2, 'y': 477.5},
  'pupilRight': {'x': 627.8, 'y': 475.4},
  'noseTip': {'x': 512.5, 'y': 646.9},
  'mouthLeft': {'x': 407.8, 'y': 763.2},
  'mouthRight': {'x': 614.1, 'y': 760.5},
  'eyebrowLeftOuter': {'x': 315.7, 'y': 439.4},
  'eyebrowLeftInner': {'x': 481.1, 'y': 434.8},
  'eyeLeftOuter': {'x': 364.1, 'y': 480.2},
  'eyeLeftTop': {'x': 398.8, 'y': 465.2},
  'eyeLeftBottom': {'x': 399.3, 'y': 494.3},
  'eyeLeftInner': {'x': 434.1, 'y': 478.1},
  'eyebrowRightInner': {'x': 566.5, 'y': 426.0},
  'eyebrowRightOuter': {'x': 724.9, 'y': 421.1},
  'eyeRightInner': {'x': 593.6, 'y': 474.9},
  'eyeRightTop': {'x': 624.7, 'y': 462.9},
  'eyeRightBottom': {'x': 625.2, 'y': 492.2},
  'eyeRightOuter': {'x': 659.5, 'y': 479.2},
  'noseRootLeft': {'x': 479.1, 'y': 486.3},
  'noseRootRight': {'x': 541.7, 'y': 483.7},
  'noseLeftAlarT

This cell is specifically for data validation. This does not need to be run if you are confident your data is valid.

In [5]:
assert(len(labels_data) != 0)
assert(labels_data[0] != {})

_dne = []
for _face in labels_data:
    if not os.path.isfile(_face["localFacePath"]):
        _dne.append(_face["localFacePath"])
if _dne:
    raise FileNotFoundError(f"The following files were not located: {', '.join(_dne)}")

print("All of the data is valid, continuing...")

All of the data is valid, continuing...


In [6]:
LATENT_TRAINING_DATA = "/home/firschj/src/CS548_Research_Code_GAN/stylegan2/backprojection/dlatent_master_file.npy"

In [7]:
dlatent_data = np.load(LATENT_TRAINING_DATA)
dlatent_data.shape

(263, 18, 512)

In [8]:
X_data = dlatent_data.reshape((-1, 18*512))
y_age_data = np.array([x['faceAttributes']['age'] for x in labels_data])
y_age_kid_data = np.array([x['faceAttributes']['age'] < 5 for x in labels_data])
y_age_young_data = np.array([x['faceAttributes']['age'] > 15  for x in labels_data])
y_age_middle_data = np.array([x['faceAttributes']['age'] > 30 for x in labels_data])
y_age_old_data = np.array([x['faceAttributes']['age'] > 60 for x in labels_data])
y_gender_data = np.array([x['faceAttributes']['gender'] == 'male' for x in labels_data]) 
y_glasses_data = np.array([x['faceAttributes']['glasses'] != 'NoGlasses' for x in labels_data])
y_eyeMakeup_data = np.array([x['faceAttributes']['makeup']['eyeMakeup'] == True for x in labels_data]) 
y_lipMakeup_data = np.array([x['faceAttributes']['makeup']['lipMakeup'] == False for x in labels_data]) 
y_headPose_roll__data = np.array([x['faceAttributes']['headPose']['roll'] < 0.8 for x in labels_data])
y_headPose_yaw__data = np.array([x['faceAttributes']['headPose']['yaw'] < 0.8 for x in labels_data])
y_smile_data = np.array([x['faceAttributes']['smile'] > 0.5 for x in labels_data])
y_anger_data = np.array([x['faceAttributes']['emotion']['anger'] > 0.1 for x in labels_data]) 
y_sadness_data = np.array([x['faceAttributes']['emotion']['sadness'] > 0.1 for x in labels_data]) 
y_contempt_data = np.array([x['faceAttributes']['emotion']['contempt'] > 0.2 for x in labels_data]) 
y_disgust_data = np.array([x['faceAttributes']['emotion']['disgust'] > 0.1 for x in labels_data]) 
y_fear_data = np.array([x['faceAttributes']['emotion']['fear'] > 0.1 for x in labels_data]) 
y_happiness_data = np.array([x['faceAttributes']['emotion']['happiness'] > 0.2 for x in labels_data]) 
y_neutral_data = np.array([x['faceAttributes']['emotion']['neutral'] > 0.2 for x in labels_data]) 
y_surprise_data = np.array([x['faceAttributes']['emotion']['surprise'] > 0.2 for x in labels_data]) 
y_beard_data = np.array([x['faceAttributes']['facialHair']['moustache'] > 0.5 for x in labels_data]) 
y_moustache_data = np.array([x['faceAttributes']['facialHair']['moustache'] > 0.5 for x in labels_data]) 
y_sideburns_data = np.array([x['faceAttributes']['facialHair']['sideburns'] > 0.5 for x in labels_data])

This cell is specifically for data validation. This does not need to be run if you are confident your data is valid.¶

In [9]:
assert(len(X_data) == len(y_age_data) == len(y_gender_data))
len(X_data)

263

In [10]:
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split
from sklearn.metrics import accuracy_score

In [11]:
TRAINED_DIRECTIONS = "trained_directions"
if not os.path.isdir(TRAINED_DIRECTIONS):
    os.makedirs(TRAINED_DIRECTIONS)

In [12]:
%%time
print("Learning Young")
clf_age_young = LogisticRegression(class_weight='balanced',solver='saga').fit(X_data, y_age_young_data)
age_young_direction = clf_age_young.coef_.reshape((18, 512))
np.save(os.path.join(TRAINED_DIRECTIONS, "age_young_direction.npy"), age_young_direction)

Learning Young
CPU times: user 2.61 s, sys: 3.1 ms, total: 2.61 s
Wall time: 2.61 s


In [13]:
%%time
print("Learning Kid")
clf_age_kid = LogisticRegression(class_weight='balanced').fit(X_data, y_age_kid_data)
age_kid_direction = clf_age_kid.coef_.reshape((18, 512))
np.save(os.path.join(TRAINED_DIRECTIONS, "age_kid_direction.npy"), age_kid_direction)

Learning Kid
CPU times: user 3.66 s, sys: 69.3 ms, total: 3.73 s
Wall time: 277 ms


In [14]:
%%time
print("Learning Middle Age")
clf_age_middle = LogisticRegression(class_weight='balanced').fit(X_data, y_age_middle_data)
age_middle_direction = clf_age_middle.coef_.reshape((18, 512))
np.save(os.path.join(TRAINED_DIRECTIONS, "age_middle_direction.npy"), age_middle_direction)

Learning Middle Age
CPU times: user 5.73 s, sys: 129 ms, total: 5.86 s
Wall time: 293 ms


In [15]:
%%time
print("Learning Old")
clf_age_old = LogisticRegression(class_weight='balanced').fit(X_data, y_age_old_data)
age_old_direction = clf_age_old.coef_.reshape((18, 512))
np.save(os.path.join(TRAINED_DIRECTIONS, "age_old_direction.npy"), age_old_direction)

Learning Old
CPU times: user 3.56 s, sys: 102 ms, total: 3.67 s
Wall time: 183 ms


In [16]:
%%time
print("Learning Gender")
clf_gender = LogisticRegression(class_weight='balanced').fit(X_data, y_gender_data)
gender_direction = clf_gender.coef_.reshape((18, 512))
np.save(os.path.join(TRAINED_DIRECTIONS, "gender_direction.npy"), gender_direction)

Learning Gender
CPU times: user 4.28 s, sys: 60.4 ms, total: 4.34 s
Wall time: 217 ms


In [17]:
%%time
print("Learning Smile")
clf_smile = LogisticRegression(class_weight='balanced').fit(X_data, y_smile_data)
smile_direction = clf_smile.coef_.reshape((18, 512))
np.save(os.path.join(TRAINED_DIRECTIONS, "smile_direction.npy"), smile_direction)

Learning Smile
CPU times: user 5.84 s, sys: 127 ms, total: 5.96 s
Wall time: 298 ms


In [18]:
%%time
print("Learning Glasses")
clf_glasses = LogisticRegression(class_weight='balanced').fit(X_data, y_glasses_data)
glasses_direction = clf_glasses.coef_.reshape((18, 512))
np.save(os.path.join(TRAINED_DIRECTIONS, "glasses_direction.npy"), glasses_direction)

Learning Glasses
CPU times: user 4.83 s, sys: 87 ms, total: 4.92 s
Wall time: 246 ms


In [19]:
%%time
print("Learning Head Yaw")
clf_headPose_yaw = LogisticRegression(class_weight='balanced').fit(X_data, y_headPose_yaw__data)
headPose_yaw_direction = clf_headPose_yaw.coef_.reshape((18, 512))
np.save(os.path.join(TRAINED_DIRECTIONS, "headPose_yaw_direction.npy"), headPose_yaw_direction)

Learning Head Yaw
CPU times: user 6.23 s, sys: 164 ms, total: 6.4 s
Wall time: 320 ms


In [20]:
%%time
print("Learning Eye Makeup")
clf_eyeMakeup = LogisticRegression(class_weight='balanced').fit(X_data, y_eyeMakeup_data)
eyeMakeup_direction = clf_eyeMakeup.coef_.reshape((18, 512))
np.save(os.path.join(TRAINED_DIRECTIONS, "eyeMakeup_direction.npy"), eyeMakeup_direction)

Learning Eye Makeup
CPU times: user 5.88 s, sys: 128 ms, total: 6.01 s
Wall time: 300 ms


In [21]:
%%time
print("Learning Lip Makeup")
clf_lipMakeup = LogisticRegression(class_weight='balanced').fit(X_data, y_lipMakeup_data)
lipMakeup_direction = clf_lipMakeup.coef_.reshape((18, 512))
np.save(os.path.join(TRAINED_DIRECTIONS, "lipMakeup_direction.npy"), lipMakeup_direction)

Learning Lip Makeup
CPU times: user 5.49 s, sys: 95.8 ms, total: 5.59 s
Wall time: 279 ms


In [22]:
%%time
print("Learning Head Roll")
clf_headPose_roll = LogisticRegression(class_weight='balanced').fit(X_data, y_headPose_roll__data)
headPose_roll_direction = clf_headPose_roll.coef_.reshape((18, 512))
np.save(os.path.join(TRAINED_DIRECTIONS, "headPose_roll_direction.npy"), headPose_roll_direction)

Learning Head Roll
CPU times: user 4.75 s, sys: 96.9 ms, total: 4.84 s
Wall time: 242 ms


In [23]:
%%time
print("Learning Anger")
clf_anger = LogisticRegression(class_weight='balanced').fit(X_data, y_anger_data)
anger_direction = clf_anger.coef_.reshape((18, 512))
np.save(os.path.join(TRAINED_DIRECTIONS, "anger_direction.npy"), anger_direction)

Learning Anger
CPU times: user 2.06 s, sys: 28.1 ms, total: 2.09 s
Wall time: 104 ms


In [24]:
%%time
print("Learning Sadness")
clf_sadness = LogisticRegression(class_weight='balanced').fit(X_data, y_sadness_data)
sadness_direction = clf_sadness.coef_.reshape((18, 512))
np.save(os.path.join(TRAINED_DIRECTIONS, "sadness_direction.npy"), sadness_direction)

Learning Sadness
CPU times: user 2.42 s, sys: 64.2 ms, total: 2.49 s
Wall time: 124 ms


In [25]:
%%time
print("Learning Contempt")
clf_contempt = LogisticRegression(class_weight='balanced').fit(X_data, y_contempt_data)
contempt_direction = clf_contempt.coef_.reshape((18, 512))
np.save(os.path.join(TRAINED_DIRECTIONS, "contempt_direction.npy"), contempt_direction)

Learning Contempt


ValueError: This solver needs samples of at least 2 classes in the data, but the data contains only one class: False

In [26]:
%%time
print("Learning Disgust")
clf_disgust = LogisticRegression(class_weight='balanced').fit(X_data, y_disgust_data)
disgust_direction = clf_disgust.coef_.reshape((18, 512))
np.save(os.path.join(TRAINED_DIRECTIONS, "disgust_direction.npy"), disgust_direction)

Learning Disgust


ValueError: This solver needs samples of at least 2 classes in the data, but the data contains only one class: False

In [27]:
%%time
print("Learning Fear")
clf_fear = LogisticRegression(class_weight='balanced').fit(X_data, y_fear_data)
fear_direction = clf_fear.coef_.reshape((18, 512))
np.save(os.path.join(TRAINED_DIRECTIONS, "fear_direction.npy"), fear_direction)

Learning Fear


ValueError: This solver needs samples of at least 2 classes in the data, but the data contains only one class: False

In [28]:
%%time
print("Learning Happy")
clf_happiness = LogisticRegression(class_weight='balanced').fit(X_data, y_happiness_data)
happiness_direction = clf_happiness.coef_.reshape((18, 512))
np.save(os.path.join(TRAINED_DIRECTIONS, "happiness_direction.npy"), happiness_direction)

Learning Happy
CPU times: user 5.6 s, sys: 127 ms, total: 5.73 s
Wall time: 286 ms


In [29]:
%%time
print("Learning Neutral")
clf_neutral = LogisticRegression(class_weight='balanced').fit(X_data, y_neutral_data)
neutral_direction = clf_neutral.coef_.reshape((18, 512))
np.save(os.path.join(TRAINED_DIRECTIONS, "neutral_direction.npy"), neutral_direction)

Learning Neutral
CPU times: user 4.69 s, sys: 71.1 ms, total: 4.76 s
Wall time: 238 ms


In [30]:
%%time
print("Learning Surprise")
clf_surprise = LogisticRegression(class_weight='balanced').fit(X_data, y_surprise_data)
surprise_direction = clf_surprise.coef_.reshape((18, 512))
np.save(os.path.join(TRAINED_DIRECTIONS, "surprise_direction.npy"), surprise_direction)

Learning Surprise


ValueError: This solver needs samples of at least 2 classes in the data, but the data contains only one class: False

In [31]:
%%time
print("Learning Beard")
clf_beard = LogisticRegression(class_weight='balanced').fit(X_data, y_beard_data)
beard_direction = clf_beard.coef_.reshape((18, 512))
np.save(os.path.join(TRAINED_DIRECTIONS, "beard_direction.npy"), beard_direction)

Learning Beard
CPU times: user 3.71 s, sys: 43.9 ms, total: 3.75 s
Wall time: 188 ms


In [32]:
%%time
print("Learning Moustache")
clf_moustache = LogisticRegression(class_weight='balanced').fit(X_data, y_moustache_data)
moustache_direction = clf_moustache.coef_.reshape((18, 512))
np.save(os.path.join(TRAINED_DIRECTIONS, "moustache_direction.npy"), moustache_direction)

Learning Moustache
CPU times: user 3.74 s, sys: 32.2 ms, total: 3.77 s
Wall time: 189 ms


In [33]:
%%time
print("Learning Sideburns")
clf_sideburns = LogisticRegression(class_weight='balanced').fit(X_data, y_sideburns_data)
sideburns_direction = clf_sideburns.coef_.reshape((18, 512))
np.save(os.path.join(TRAINED_DIRECTIONS, "sideburns_direction.npy"), sideburns_direction)

Learning Sideburns
CPU times: user 2.7 s, sys: 32.3 ms, total: 2.74 s
Wall time: 137 ms
